## Example notebook to run ParaSweeper

### <a id='toc1_1_1_'></a>[Functions to be achieved:](#toc0_)
1. reduce ram used; 
2. reduce code simplicity by building new class; (class: Controller; Path; )
3. save more frequently
4. reduce test time

In [88]:
# magic lines that avoid re-start 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [89]:
# Load modules
import pybamm as pb;import pandas as pd;import numpy as np;
import os, json,openpyxl,traceback,multiprocessing,scipy.optimize,sys,gc
import matplotlib.pyplot as plt;
import pickle,imageio,timeit,random,time, signal
from scipy.io import savemat,loadmat;
from pybamm import constants,exp;import matplotlib as mpl

In [90]:
str_path_0 = os.path.abspath(os.path.join(pb.__path__[0],'..'))
str_path_1 = os.path.abspath(
    os.path.join(str_path_0,"/ParaSweeper"))
str_path_1

'/ParaSweeper'

In [91]:
# reduce variables that require define outside
On_HPC = False
if On_HPC:
    case_no = int(os.environ["PBS_ARRAY_INDEX"])
    Path_Input = "InputData/" 
    BasicPath_Save=os.getcwd() 
else:
    str_path_0 = os.path.abspath(os.path.join(pb.__path__[0],'..'))
    str_path_1 = os.path.abspath(
        os.path.join(str_path_0,"ParaSweeper"))
    sys.path.append(str_path_1) 
    
    case_no = 6
    #### Note: you will need to change these two paths to fit your system. 
    Path_Input = os.path.expanduser(
        "~/EnvPBGEM_NC/SimSave/InputData/") # for experimental data, like Niall's ageing experiment
    BasicPath_Save =  os.path.expanduser(
        "~/EnvPBGEM_NC/SimSave/P2_R9_Dim") # path to save simulation results
# import all functions 
from main import run_one_case
from main import *
from TaskResult import *
from TaskConfig import *
from get_input_para import *

In [92]:
# define class for all inputs and customized settings:
# start to do configuration:
path_config = PathConfig(
    On_HPC=On_HPC, Path_Input=Path_Input,
    BasicPath_Save=BasicPath_Save,
    purpose_i="Full_Exp1235_NC", case_no=case_no,
    rows_per_file= 1,Scan_end_end= 12,
    )
global_config = GlobalConfig(
    On_HPC=On_HPC, Runshort=True, 
    Plot_Exp=True, Timeout=True, Return_Sol=True, 
    Check_Short_Time=True, R_from_GITT=True,
    fs=13, dpi=100, Re_No=0, Timelimit=int(3600*48),
    Timeout_text = 'I timed out'   )

exp_config = ExpConfig()
para_config = ParameterConfig()
model_config = ModelConfig()
expData_config = ExperimentDataConfig(cap_0 = 4.86491)
config = TaskConfig(
    path_config=path_config, global_config=global_config, 
    exp_config=exp_config,para_config=para_config,
    model_config=model_config,expData_config=expData_config, Scan_No=case_no)

# Load input file

# normal version
# Para_dict_list = load_combinations_from_csv(config.path_config.Para_file) 

# for demo purpose only
path_csv_special_for_test = str_path_1 + "/Bundle_6.csv"
Para_dict_list = load_combinations_from_csv(path_csv_special_for_test) 
pool_no = len(Para_dict_list) # do parallel computing if needed

Total scan case is 1


In [93]:
# hack the input to make demo quicker:
Para_dict_list[0]['Mesh list'] = '[5,5,5,60,20]'
Para_dict_list[0]["Negative electrode diffusivity [m2.s-1]"] = 3.3e-14
Para_dict_list[0]["Positive electrode diffusivity [m2.s-1]"] = 4e-15
Para_dict_list[0]['Exp No.'] = 6 # in this case, no experiment data needed
Para_dict_list[0]

{'Scan No': 6,
 'Exp No.': 6,
 'Ageing temperature': 40,
 'Mesh list': '[5,5,5,60,20]',
 'Inner SEI lithium interstitial diffusivity [m2.s-1]': 9.81e-19,
 'Dead lithium decay constant [s-1]': 1e-07,
 'Lithium plating kinetic rate constant [m.s-1]': 1e-10,
 'Negative electrode LAM constant proportional term [s-1]': 2.84e-09,
 'Positive electrode LAM constant proportional term [s-1]': 2.98e-18,
 'Negative electrode cracking rate': 5.29e-25,
 'Outer SEI partial molar volume [m3.mol-1]': 5.22e-05,
 'SEI growth activation energy [J.mol-1]': 5000.0,
 'Negative cracking growth activation energy [J.mol-1]': 0.0,
 'Negative electrode diffusivity activation energy [J.mol-1]': 20000.0,
 'Positive electrode diffusivity activation energy [J.mol-1]': 12000.0,
 'Initial electrolyte excessive amount ratio': 1.0,
 'Cycles within RPT': 1,
 'RPT temperature': 25,
 'Para_Set': 'OKane2023',
 'Model option': "{'SEI': 'interstitial-diffusion limited', 'SEI on cracks': 'true', 'lithium plating': 'partially re

In [94]:

# Assuming we only do the first one:
config.get_para_dictionary(Para_dict_list[0])

# config.para_config.Para_dict_i['Exp No.'] = 8

config.get_expconfig()
config.initialize_experiment_text()
config.get_exp_data_config()

Aging T is 40degC


### Run the model

In [95]:
# Run the model, assuming only one case:
if config.global_config.Re_No == 0:
    task_result, config = run_one_case(config) 
elif config.global_config.Re_No > 0:
    pass


Start Now! Scan 6 Re 0
Has Overwritten Initial outer SEI thickness [m] to be 
        9.23e-09 and Negative electrode 
        porosity to be 0.235 to account for initial 
        capacity loss of 0.135 Ah
Initial electrolyte amount is  5.236163896909271 mL
Scan 6 Re 0: Spent 1.417 ms on Initialization, DryOut = On



Succeed to run break in for the 0th time with perturbation of 0.00e+00Ah

Scan 6 Re 0: Finish break-in cycle within 50.111 s
Scan 6 Re 0: Finish post-process for break-in cycle within 1.947 s
Electrolyte is being squeezed out, check plated lithium (active and dead)


At t = 314.598 and h = 6.61298e-12, the corrector convergence failed repeatedly or with |h| = hmin.


Succeed to run the ageing set for 1 cycles the 1th time
Scan 6 Re 0: Finish No.
              0 
              ageing cycles within 19.069 s
Scan 6 Re 0: Finish post-process for 
              No. 1 
              ageing cycles within 155.827 ms
Succeed to run RPT for the 1th time
Scan 6 Re 0: Finish No."
            1"
            RPT cycles within 44.934 s
Scan 6 Re 0: Finish post-process for No.
            1 
            RPT cycles within 2.223 s
Scan 6 Re 0: Getting extra variables within 1.950 ms
Scan 6 Re 0: Finish all plots within 5.488 s
Electrolyte is being squeezed out, check plated lithium (active and dead)
Scan 6 Re 0: Save Early for RPT 0 within 439.347 ms


At t = 314.26 and h = 2.71212e-11, the corrector convergence failed repeatedly or with |h| = hmin.


Succeed to run the ageing set for 1 cycles the 1th time
Scan 6 Re 0: Finish No.
              1 
              ageing cycles within 15.715 s
Scan 6 Re 0: Finish post-process for 
              No. 2 
              ageing cycles within 178.859 ms
Succeed to run RPT for the 1th time
Scan 6 Re 0: Finish No."
            2"
            RPT cycles within 41.911 s
Scan 6 Re 0: Finish post-process for No.
            2 
            RPT cycles within 2.433 s
Scan 6 Re 0: Getting extra variables within 1.088 ms
Scan 6 Re 0: Finish all plots within 5.441 s
Electrolyte is being squeezed out, check plated lithium (active and dead)
Scan 6 Re 0: Save Early for RPT 1 within 445.082 ms
Scan 6 Re 0: Finish all plots within 5.436 s
Scan 6 Re 0: Try saving within 60.676 ms
Succeed doing something in 3 minutes, 16 seconds
This is the end of No. 6 scan, Re 0
